# 1. Basic data storage

All programs process data in one form or another, and many need to be able to save and retrieve that data from one invocation to the next. In `Python`, we have many ways to store data. 
- Flat files (e.g. csv, json, xml, etc.)
- RDBMS (e.g. postgres, mysql, etc)
- NoSQL DB (e.g. mongoDB, Neo4J, etc.)

In this tutorial, we will use first flat files, then RDBMS (SQLite), to store and retrieve data. At last we will use `SQLAlchemy (ORM)` which allow you to map rows in a table with python object. `SQLite` is a light weight RDBMS which stores data in a single file without the need for complex installation.

## 1.1 Working with flat files 

### Advantages of Flat Files
- Easy to use and implement
- human-readable makes it easy to edit, view/examine, and transfer 
- support by many language and framework (import, export)


### Disadvantages of Flat Files
- performance: Large files are difficult to view/examine/edit
- No explicit relation between data parts (tables). Relation need to be written in the business logic
- No schema or data structure. Users need understand not only the structure of the data but also the programming tools necessary for accessing it.